In [11]:
import os
import re
import pandas as pd
from tqdm import tqdm

# ============================
# 0. PATH 설정 (대장 환경에 맞게 조정)
# ============================
RAW_DIR = "../../data/raw"        # 🔴 원본 train/dev/test 위치
OUT_DIR = "../../data/processed"  # 🔵 전처리된 파일 저장 위치

os.makedirs(OUT_DIR, exist_ok=True)

RAW_FILES = {
    "train": "train.csv",
    "dev": "dev.csv",
    "test": "test.csv",
}

# ============================
# 1. 유틸 함수들
# ============================
def normalize_newlines(text: str) -> str:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = text.replace("\\n", "\n")
    text = re.sub(r"<br\s*/?>", "\n", text, flags=re.I)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text

def normalize_quotes_symbols(text: str) -> str:
    rep = {
        "“": '"', "”": '"',
        "‘": "'", "’": "'",
        "「": '"', "」": '"',
        "『": '"', "』": '"'
    }
    for k, v in rep.items():
        text = text.replace(k, v)
    text = re.sub(r"[『』《》─━│┃╭╮╰╯]", "", text)
    return text

# 스피커 태그: "#PersonN#" 이면서 바로 뒤에 공백 또는 콜론이 오는 경우만 "턴 시작"으로 본다.
SPEAKER_TAG = re.compile(r"(#Person\d+#)(?=\s|:)", re.I)

def split_by_speaker(text: str):
    """
    1) "#PersonN#" 앞에 줄바꿈을 넣어서
    2) 각 줄의 맨 앞 "#PersonN#"을 화자로 인식
    3) 문장 중간의 "#Person1#님" 같은 건 건드리지 않음
    """
    # "#PersonN#" 앞에 newline 삽입 (턴 시작 후보만)
    text = SPEAKER_TAG.sub(r"\n\1", text).strip()

    lines = [ln.strip() for ln in text.split("\n") if ln.strip()]
    segments = []
    current_spk = None
    current_utt = []

    for ln in lines:
        m = SPEAKER_TAG.match(ln)
        if m:
            # 새로운 화자 등장
            if current_spk is not None and current_utt:
                segments.append((current_spk, " ".join(current_utt).strip()))
            current_spk = m.group(1)  # "#Person1#"
            rest = ln[m.end():].lstrip(" :")
            current_utt = [rest] if rest else []
        else:
            # 이전 화자의 발화 이어붙이기
            if current_spk is None:
                # 맨 처음에 speaker 없이 텍스트가 나오면 그냥 하나의 utterance로 취급
                current_spk = ""
                current_utt = [ln]
            else:
                current_utt.append(ln)

    if current_spk is not None and current_utt:
        segments.append((current_spk, " ".join(current_utt).strip()))

    return segments


def clean_utterance(text: str) -> str:
    t = text.strip()

    # 1) 감정 표현 제거
    t = re.sub(r"(ㅋㅋ+|ㅎㅎ+|ㅠㅠ+|ㅜㅜ+)", "", t)

    # 2) 물결 제거 또는 "." 통일
    t = re.sub(r"~{2,}", ".", t)

    # 3) 말줄임표 "." 하나로 통일
    t = re.sub(r"\.{2,}", ".", t)

    # 4) 반복 단어 3회 이상 제거
    t = re.sub(r"\b(\w+)(\s+\1){2,}\b", r"\1", t)

    # 5) 문장부호 조합 정리
    t = re.sub(r",\?", "?", t)
    t = re.sub(r"\?+", "?", t)
    t = re.sub(r"!+", "!", t)
    t = re.sub(r"\?\.","?", t)
    t = re.sub(r"!\.","!", t)

    # 6) 공백 정리
    t = re.sub(r"\s+([?.!,])", r"\1", t)
    t = re.sub(r"\s+", " ", t).strip()

    # --------------------------------------------
    # 7) 문장 끝 마침표 보정 (핵심)
    # --------------------------------------------
    # 끝이 .,?,! 로 안 끝나면 "." 추가
    if t and t[-1] not in ".?!":
        t = t + "."

    return t


def preprocess_dialogue(text: str) -> str:
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    t = normalize_newlines(text)
    t = normalize_quotes_symbols(t)

    segments = split_by_speaker(t)

    # 스피커 태그를 하나도 못 찾으면 그냥 클린업만 해서 반환
    if not segments:
        return clean_utterance(t)

    lines = []
    for spk, utt in segments:
        cu = clean_utterance(utt)
        if not cu:
            continue
        if spk:
            lines.append(f"{spk}: {cu}")
        else:
            lines.append(cu)

    return "\n".join(lines)

# ============================
# 2. 실제 전처리 실행
# ============================
for split, fname in RAW_FILES.items():
    in_path = os.path.join(RAW_DIR, fname)
    out_path = os.path.join(OUT_DIR, f"v3_{split}_preprocessed.csv")

    print(f"[INFO] Preprocessing {split.upper()}  ({in_path} -> {out_path})")
    df = pd.read_csv(in_path)

    # 원본은 'dialogue' 컬럼 기준 (대회 포맷)
    df["dialogue"] = [preprocess_dialogue(t) for t in tqdm(df["dialogue"])]

    df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("[INFO] All done!")
print(os.path.join(OUT_DIR, "v3_train_preprocessed.csv"))
print(os.path.join(OUT_DIR, "v3_dev_preprocessed.csv"))
print(os.path.join(OUT_DIR, "v3_test_preprocessed.csv"))

# ============================
# 3. 샘플 확인
# ============================
sample = """
#Person1#: 안녕하세요!! 피자 주문하실래요??.
#Person2#: 네, #Person1#님. 페퍼로니랑 올리브 추가요!! ㅎㅎㅎ
#Person1#: 대형 피자 할인 있어요~~~ 결국 결국 결국
#Person2#: 오 그런가요,?? 그럼 대형 주세요ㅋㅋㅋㅋㅋㅋㅋ
"""

print("=== BEFORE SAMPLE ===")
print(sample)
print("\n=== AFTER SAMPLE (preprocessed) ===")
print(preprocess_dialogue(sample))


[INFO] Preprocessing TRAIN  (../../data/raw/train.csv -> ../../data/processed/v3_train_preprocessed.csv)


100%|██████████| 12457/12457 [00:02<00:00, 5805.41it/s]


[INFO] Preprocessing DEV  (../../data/raw/dev.csv -> ../../data/processed/v3_dev_preprocessed.csv)


100%|██████████| 499/499 [00:00<00:00, 6226.11it/s]


[INFO] Preprocessing TEST  (../../data/raw/test.csv -> ../../data/processed/v3_test_preprocessed.csv)


100%|██████████| 499/499 [00:00<00:00, 6025.74it/s]

[INFO] All done!
../../data/processed/v3_train_preprocessed.csv
../../data/processed/v3_dev_preprocessed.csv
../../data/processed/v3_test_preprocessed.csv
=== BEFORE SAMPLE ===

#Person1#: 안녕하세요!! 피자 주문하실래요??.
#Person2#: 네, #Person1#님. 페퍼로니랑 올리브 추가요!! ㅎㅎㅎ
#Person1#: 대형 피자 할인 있어요~~~ 결국 결국 결국
#Person2#: 오 그런가요,?? 그럼 대형 주세요ㅋㅋㅋㅋㅋㅋㅋ


=== AFTER SAMPLE (preprocessed) ===
#Person1#: 안녕하세요! 피자 주문하실래요?
#Person2#: 네, #Person1#님. 페퍼로니랑 올리브 추가요!
#Person1#: 대형 피자 할인 있어요. 결국.
#Person2#: 오 그런가요? 그럼 대형 주세요.
